# AAPL Multi-Expiry Demo: VolSurface + ProbSurface

This notebook extends the AAPL example to multiple expiries. We fit a
VolSurface, slice it into curves, and build a ProbSurface for a cross-expiry
probability view.

Goals:
- Fit a VolSurface to multiple expiries
- Query implied vol and total variance across time
- Slice the surface into VolCurve snapshots
- Build a ProbSurface and plot a fan chart

## Setup

In [ ]:
from datetime import date

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from oipd import VolSurface, MarketInputs, sources

## Load the AAPL chain

We keep multiple expiries (within a horizon) to fit a surface.

In [ ]:
COLUMN_MAPPING = {
    "strike": "strike",
    "last_price": "last_price",
    "bid": "bid",
    "ask": "ask",
    "expiration": "expiry",
    "type": "option_type",
}

df_aapl = sources.from_csv("data/AAPL_data.csv", column_mapping=COLUMN_MAPPING)

## Market inputs

In [ ]:
market = MarketInputs(
    valuation_date=date(2025, 10, 6),
    risk_free_rate=0.04,
    underlying_price=256.69,
)
market

## Fit the VolSurface

We use a 3-month horizon to keep the example light. You can remove the horizon
argument to fit all expiries in the CSV.

In [ ]:
vol_surface = VolSurface(method="svi", pricing_engine="black76")
vol_surface.fit(df_aapl, market, horizon="3m")

vol_surface.expiries

## Query the surface

Choose a representative expiry and convert it into time-to-maturity (years).

In [ ]:
sample_expiry = vol_surface.expiries[0]

# Convert expiry to years for functions that take t
# We use a simple ACT/365 day count for demo purposes
t_years = (sample_expiry.date() - market.valuation_date).days / 365.0

strikes = np.array([220.0, 240.0, 260.0, 280.0, 300.0])

iv_atm = vol_surface.implied_vol(market.underlying_price, t_years)
forward = vol_surface.forward_price(t_years)

{
    "sample_expiry": sample_expiry,
    "t_years": t_years,
    "forward_price": forward,
    "atm_vol": vol_surface.atm_vol(sample_expiry),
    "iv_atm": iv_atm,
}

In [ ]:
# Prices and Greeks at the chosen expiry
call_prices = vol_surface.price(strikes, t=sample_expiry, call_or_put="call")
all_greeks = vol_surface.greeks(strikes, t=sample_expiry, call_or_put="call")
all_greeks["call_price"] = call_prices
all_greeks

## Surface diagnostics and plots

In [ ]:
vol_surface.iv_results().head()

In [ ]:
vol_surface.plot()
plt.show()

In [ ]:
vol_surface.plot_3d()
plt.show()

In [ ]:
vol_surface.plot_term_structure()
plt.show()

## Slice the surface into VolCurves

Exact expiries return the fitted parametric slice. Interpolated expiries return
synthetic curves derived from total-variance interpolation.

In [ ]:
# Exact pillar slice
pillar_curve = vol_surface.slice(sample_expiry)

# Interpolated slice between first two expiries
exp0 = vol_surface.expiries[0]
exp1 = vol_surface.expiries[1]
interp_expiry = exp0 + (exp1 - exp0) / 2
interp_curve = vol_surface.slice(interp_expiry)

{
    "pillar_expiry": exp0,
    "interp_expiry": interp_expiry,
}

In [ ]:
pillar_curve.plot(include_observed=True)
plt.show()

In [ ]:
interp_curve.plot(include_observed=False)
plt.show()

## ProbSurface

We can build a cross-expiry probability view and then slice it like a surface.

In [ ]:
prob_surface = vol_surface.implied_distribution()
prob_surface.expiries[:3]

In [ ]:
prob_surface.plot_fan(lower_percentile=10.0, upper_percentile=90.0)
plt.show()

In [ ]:
prob_surface.slice(sample_expiry).plot(kind="both")
plt.show()

## Wrap-up

You have now:
- A fitted VolSurface with time interpolation
- The ability to slice into VolCurves
- A ProbSurface with fan-chart visualization

Next: see the quickstart notebook for live yfinance data.